In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [2]:
rd_velomagg = pd.read_csv("../data/video/courses/TAM_MMM_CoursesVelomagg.csv", encoding="utf-8-sig")  

In [3]:
def correct_encoding(value):
    if isinstance(value, str):
        return value.encode("latin1", errors="replace").decode("utf-8", errors="replace")
    return value

In [4]:
rd_velomagg = rd_velomagg.apply(lambda col: col.apply(correct_encoding) if col.dtype == "object" else col)

In [5]:
rd_velomagg["Date"] = pd.to_datetime(rd_velomagg["Departure"]).dt.date

In [6]:
date_debut = pd.to_datetime("2024-03-18").date()
date_fin = pd.to_datetime("2024-03-24").date()

In [7]:
rd_velomagg = rd_velomagg[(rd_velomagg["Date"] >= date_debut) & (rd_velomagg["Date"] <= date_fin)]
dates = pd.date_range(date_debut, date_fin).date
weekdays = {date.strftime("%A"): date for date in dates}

In [8]:
def plot_histogram(jour):
    date = weekdays[jour]

    rd_velo_par_jour = rd_velomagg[rd_velomagg["Date"] == date]

    r_sum = rd_velo_par_jour["Departure station"].value_counts()
    d_sum = rd_velo_par_jour["Return station"].value_counts()

    stations = sorted(set(r_sum.index).union(set(d_sum.index)))
    r_sum = r_sum.reindex(stations, fill_value=0)
    d_sum = d_sum.reindex(stations, fill_value=0)

    x = range(len(stations))

    plt.figure(figsize=(12, 6))
    plt.bar(x, r_sum, width=0.4, label="Retrait", color="plum")
    plt.bar([i + 0.4 for i in x], d_sum, width=0.4, label="Dépôt", color="magenta")

    plt.xlabel("Stations")
    plt.ylabel("Nombre de retraits ou dépôts")
    plt.title(f"Retraits et dépôts des vélomaggs le {jour}")
    plt.xticks([i + 0.4 / 2 for i in x], stations, rotation=45, ha="right")
    plt.legend()

    plt.tight_layout()
    plt.show()

In [9]:
dropdown = Dropdown(
    options=list(weekdays.keys()),
    description="Jour de la semaine : ",
    value=list(weekdays.keys())[0],
    style={'description_width': 'initial'}
)

In [10]:
graph = interact(plot_histogram, jour=dropdown)
graph

interactive(children=(Dropdown(description='Jour de la semaine : ', options=('Lundi', 'Mardi', 'Mercredi', 'Je…

<function __main__.plot_histogram(jour)>

In [11]:
#plt.offline.plot(graph, filename = 'rd_velomagg_graph.html', auto_open=False)